# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Zieba  ->  S. Zieba  |  ['S. Zieba']


S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Arxiv has 65 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.04870
extracting tarball to tmp_2406.04870... done.


Found 110 bibliographic references in tmp_2406.04870/main_arxiv_submission_june5.bbl.
Retrieving document from  https://arxiv.org/e-print/2406.05118


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Burrows} {et~al.}(1995){Burrows}, {Krist}, {Stapelfeldt},  {WFPC2  Investigation Definition Team}}]{Burrows1995}{Burrows}, C.~J., {Krist}, J.~E., {Stapelfeldt}, K.~R.,  {WFPC2 Investigation  Definition Team}. 1995, in American Astronomical Society Meeting Abstracts,  Vol. 187, American Astronomical Society Meeting Abstracts, 32.05
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


extracting tarball to tmp_2406.05118...

 done.


S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure J2020_emupilot
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure J2020_emupilot as tmp_2406.05118/./J2020_emupilot.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 192 bibliographic references in tmp_2406.05118/aanda.bbl.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{HI4PI Collaboration} {et~al.}(2016){HI4PI Collaboration}, {Ben Bekhti}, {Fl{ö}er}, {Keller}, {Kerp}, {Lenz}, {Winkel}, {Bailin}, {Calabretta}, {Dedes}, {Ford}, {Gibson}, {Haud}, {Janowiecki}, {Kalberla}, {Lockman}, {McClure-Griffiths}, {Murphy}, {Nakanishi}, {Pisano},  {Staveley-Smith}}]{hi4pi16}{HI4PI Collaboration}, {Ben Bekhti}, N., {Fl{ö}er}, L., {et~al.} 2016, \aap, 594, A116
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.04870-b31b1b.svg)](https://arxiv.org/abs/2406.04870) | **The $\beta$ Pictoris b Hill sphere transit campaign. Paper II: Searching for the signatures of the $\beta$ Pictoris exoplanets through time delay analysis of the $\delta$ Scuti pulsations**  |
|| <mark>S. Zieba</mark>, et al. |
|*Appeared on*| *2024-06-10*|
|*Comments*| *16 pages, 16 figures, 4 tables, accepted for publication in A&A*|
|**Abstract**|            The $\beta$ Pictoris system is the closest known stellar system with directly detected gas giant planets, an edge-on circumstellar disc, and evidence of falling sublimating bodies and transiting exocomets. The inner planet, $\beta$ Pictoris c, has also been indirectly detected with radial velocity (RV) measurements. The star is a known $\delta$ Scuti pulsator, and the long-term stability of these pulsations opens up the possibility of indirectly detecting the gas giant planets through time delays of the pulsations due to a varying light travel time. We search for phase shifts in the $\delta$ Scuti pulsations consistent with the known planets $\beta$ Pictoris b and c and carry out an analysis of the stellar pulsations of $\beta$ Pictoris over a multi-year timescale. We used photometric data collected by the BRITE-Constellation, bRing, ASTEP, and TESS to derive a list of the strongest and most significant $\delta$ Scuti pulsations. We carried out an analysis with the open-source python package maelstrom to study the stability of the pulsation modes of $\beta$ Pictoris in order to determine the long-term trends in the observed pulsations. We did not detect the expected signal for $\beta$ Pictoris b or $\beta$ Pictoris c. The expected time delay is 6 seconds for $\beta$ Pictoris c and 24 seconds for $\beta$ Pictoris b. With simulations, we determined that the photometric noise in all the combined data sets cannot reach the sensitivity needed to detect the expected timing drifts. An analysis of the pulsational modes of $\beta$ Pictoris using maelstrom showed that the modes themselves drift on the timescale of a year, fundamentally limiting our ability to detect exoplanets around $\beta$ Pictoris via pulsation timing.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.05118-b31b1b.svg)](https://arxiv.org/abs/2406.05118) | **The SRG/eROSITA All-Sky Survey: X-ray beacons at late cosmic dawn**  |
|| J. Wolf, et al. -- incl., <mark>S. Belladitta</mark> |
|*Appeared on*| *2024-06-10*|
|*Comments*| *Submitted to A&A, June 7, 2024*|
|**Abstract**|            The SRG/eROSITA All-Sky Survey (eRASS) is expected to contain ~100 quasars that emitted their light when the universe was less than a billion years old, i.e. at z>5.6. By selection, these quasars populate the bright end of the AGN X-ray luminosity function and their count offers a powerful demographic diagnostic of the parent super-massive black hole population. Of the >~ 400 quasars that have been discovered at z>5.6 to date, less than 15 % have been X-ray detected. We present a pilot survey to uncover the elusive X-ray luminous end of the distant quasar population. We have designed a quasar selection pipeline based on optical, infrared and X-ray imaging data from DES DR2, VHS DR5, CatWISE2020 and the eRASS. The core selection method relies on SED template fitting. We performed optical follow-up spectroscopy with the Magellan/LDSS3 instrument for the redshift confirmation of a subset of candidates. We have further obtained a deeper X-ray image of one of our candidates with Chandra ACIS-S. We report the discovery of five new quasars in the redshift range 5.6 < z < 6.1. Two of these quasars are detected in eRASS and are by selection X-ray ultra-luminous. These quasars are also detected at radio frequencies. The first one is a broad absorption line quasar which shows significant X-ray dimming over 3.5 years, i.e. about 6 months in the quasar rest frame. The second radio-detected quasar is a jetted source with compact morphology. We show that a blazar configuration is likely for this source, making it the second most distant blazar known to date. With our pilot study, we demonstrate the power of eROSITA as a discovery machine for luminous quasars in the epoch of reionization. The X-ray emission of the two eROSITA detected quasars are likely to be driven by different high-energetic emission mechanisms a diversity which will be further explored in a future systematic full-hemisphere survey.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2406.04870.md
    + _build/html/tmp_2406.04870/./final_figs/final1.png
    + _build/html/tmp_2406.04870/./final_figs/final2.png
    + _build/html/tmp_2406.04870/./final_figs/TD_ecc_sim1.png
exported in  _build/html/2406.05118.md
    + _build/html/tmp_2406.05118/./pflux.png
    + _build/html/tmp_2406.05118/./chandra_det.png
    + _build/html/tmp_2406.05118/./lxz_simple.png
    + _build/html/tmp_2406.05118/./aoxluv_simple.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\bpic}{\beta Pictoris}$
$\newcommand{\bpicb}{\beta Pictoris b}$
$\newcommand{\bpicc}{\beta Pictoris c}$
$\newcommand{\dsct}{\delta~Scuti }$
$\newcommand{\gdor}{\gamma~Doradus }$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\MJ}{M_{J}}$
$\newcommand{\RJ}{R_{J}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Teff}{\ensuremath{T_{\mathrm{eff}}}}$
$\newcommand{\cd}{ d^{\rm -1}}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\kms}{\mathrm{km s}^{-1}}$</div>



<div id="title">

# The $\beta$ Pictoris b Hill sphere transit campaign

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.04870-b31b1b.svg)](https://arxiv.org/abs/2406.04870)<mark>Appeared on: 2024-06-10</mark> -  _16 pages, 16 figures, 4 tables, accepted for publication in A&A_

</div>
<div id="authors">

<mark>S. Zieba</mark>, et al.

</div>
<div id="abstract">

**Abstract:** The $\bpic$ system is the closest known stellar system with directly detected gas giant planets, an edge-on circumstellar disc, and evidence of falling sublimating bodies and transiting exocomets.   The inner planet, $\bpicc$ , has also been indirectly detected with radial velocity (RV) measurements.   The star is a known $\dsct$ pulsator, and the long-term stability of these pulsations opens up the possibility of indirectly detecting the gas giant planets through time delays of the pulsations due to a varying light travel time.    We search for phase shifts in the $\dsct$ pulsations consistent with the known planets $\bpicb$ and c and carry out an analysis of the stellar pulsations of $\beta$ Pictoris over a multi-year timescale.   We used photometric data collected by the BRITE-Constellation, bRing, ASTEP, and TESS to derive a list of the strongest and most significant $\dsct$ pulsations.   We carried out an analysis with the open-source python package \texttt{maelstrom} to study the stability of the pulsation modes of $\beta$ Pictoris in order to determine the long-term trends in the observed pulsations.   We did not detect the expected signal for $\bpicb$ or $\bpicc$ .   The expected time delay is 6 seconds for $\bpicc$ and 24 seconds for $\bpicb$ .    With simulations, we determined that the photometric noise in all the combined data sets cannot reach the sensitivity needed to detect the expected timing drifts.    An analysis of the pulsational modes of $\beta$ Pictoris using \texttt{maelstrom} showed that the modes themselves drift on the timescale of a year, fundamentally limiting our ability to detect exoplanets around $\beta$ Pictoris via pulsation timing.

</div>

<div id="div_fig1">

<img src="tmp_2406.04870/./final_figs/final1.png" alt="Fig11" width="100%"/>

**Figure 11. -** Time delay plot calculated from the phases of four different frequencies for all available observations by BRITE, bRing, ASTEP and TESS. Each color represents a frequency (f1, f2, f3, and f4) listed in Table \ref{tab:freqs}.
The blue lines indicate time delay predictions for $\bpic$b(dashed line), c (dashed, dotted line), and both planets (solid line).
The lower panel is a zoom-in of the upper panel.
The uncertainties in the time delays were derived from the covariance matrices given by the least-squares procedure, which was used in order to calculate the phases of the respective frequencies.
The ticks at the top of the plot denote the various observatories: B15, B16, and B17 for the BRITE observations in the years 2015, 2016, and 2017, respectively; A17 and A18 for the ASTEP observations in 2017 and 2018; and R1 and R2 for the first and second part of the bRing data. (*fig:finall*)

</div>
<div id="div_fig2">

<img src="tmp_2406.04870/./final_figs/final2.png" alt="Fig12" width="100%"/>

**Figure 12. -** Time delay plot for the simulated data set showing a high similarity to the time delays of the real data set presented in Figure \ref{fig:final2}.
The colored points represent the simulated time delays for the four strongest pulsational frequencies.
The blue lines indicate time delay predictions for $\bpic$b(dashed line), c (dashed-dotted line), and both planets (solid line).
The lower panel is a zoom-in of the upper panel.
A description of the ticks at the top of the plot can be found in the caption of Figure \ref{fig:finall}. (*fig:final2*)

</div>
<div id="div_fig3">

<img src="tmp_2406.04870/./final_figs/TD_ecc_sim1.png" alt="Fig3" width="100%"/>

**Figure 3. -** Simulation of a companion in a circular (\(e = 0\); left column) and eccentric (\(e = 0.9\); right column) orbit as if observed by the Kepler Space Telescope.
The following parameters were used: \(P = 1\) year, $\varpi = 0$, \(M_\text{pulsating} = 1.8 \)$\Msun$ , and \(M_\text{companion} = 0.1 \)$\Msun$.
This led to a semi-amplitude of around 34 seconds in the circular case (using Equation \ref{eq:TD_period}) and around 15 seconds in the eccentric case (using Equation \ref{eq:TD_eccentric}). _Upper panel:_ Simulated time delay plot.
_Lower panel:_ Fourier transformation of the time delays.
One can clearly see the relative increase of the first harmonic at two cycles/year for the eccentric case. (*fig:TDsim*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.04870"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mockalph}[1]$</div>



<div id="title">

# The SRG/eROSITA All-Sky Survey: \  X-ray beacons at late cosmic dawn

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.05118-b31b1b.svg)](https://arxiv.org/abs/2406.05118)<mark>Appeared on: 2024-06-10</mark> -  _Submitted to A&A, June 7, 2024_

</div>
<div id="authors">

J. Wolf, et al. -- incl., <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** The Spectrum Roentgen Gamma (SRG)/eROSITA All-Sky Survey (eRASS) is expected to contain $\sim100$ quasars that emitted their light when the universe was less than a billion years old, i.e. at $z>5.6$ . By selection, these quasars populate the bright end of the AGN X-ray luminosity function and their count offers a powerful demographic diagnostic of the parent super-massive black hole population. Of the $\gtrapprox 400$ quasars that have been discovered at $z>5.6$ to date, less than 15 \% have been X-ray detected. We present a pilot survey to uncover the elusive X-ray luminous end of the distant quasar population. We have designed a quasar selection pipeline based on optical, infrared and X-ray imaging data from DES DR2, VHS DR5, CatWISE2020 and the eRASS (up to eRASS:4).   The core selection method relies on SED template fitting. We performed optical follow-up spectroscopy with the Magellan/LDSS3 instrument for the redshift confirmation of a subset of candidates. We have further obtained a deeper X-ray image of one of our candidates with Chandra ACIS-S. We report the discovery of five new quasars in the redshift range $5.6 < z < 6.1$ . Two of these quasars are detected in eRASS and are by selection X-ray ultra-luminous. These quasars are also detected at radio frequencies. The first one is a broad absorption line quasar which shows significant, order-of-magnitude X-ray dimming over 3.5 years, i.e. about 6 months in the quasar rest frame. The second radio-detected quasar is a jetted source with compact morphology. We show that a blazar configuration is likely for this source, making it the second most distant blazar known to date. With our pilot study, we demonstrate the power of eROSITA as a discovery machine for luminous quasars in the epoch of reionization. The X-ray emission of the two eROSITA detected quasars are likely to be driven by different high-energetic emission mechanisms a diversity which will be further explored in a future systematic full-hemisphere survey.

</div>

<div id="div_fig1">

<img src="tmp_2406.05118/./pflux.png" alt="Fig5" width="100%"/>

**Figure 5. -** _Upper panel_: The cumulative distribution of expected random (unrelated) X-ray detections from forced photometry using a circular aperture of radius 30" around 1604 optical quasar candidate positions with. The black curve is the estimate obtained from Eq. 2. for a single eRASS survey. The black dashed curve shows an empirical estimate of the same number based on a random sample of positions in the surveyed field. The brown curve shows the number of expected contaminants obtained from a random sample in eRASS:4. Blue vertical lines indicate the $P_B$ values of the best detection of quasars discovered in this work in any eRASS and in eRASS:4 (if available). _Lower panel_: Fraction of background sources $f_{\rm bkg}$ expected to be detected in eRASS1 from a simulation by  [Seppi, Comparat and Bulbul (2022)](). (*fig:spurious*)

</div>
<div id="div_fig2">

<img src="tmp_2406.05118/./chandra_det.png" alt="Fig12" width="100%"/>

**Figure 12. -** All images are centered on the optical position of the new quasar J202040-621509 and have sizes 100' x100". _Left panel_: Broadband (0.5-7 keV) Chandra image. Chandra sources detected with 30" (black, dashed circle) of the quasar coordinates with \texttt{wavdetect} are marked in red. The centroid of 1eRASS J202039.8-621525 catalogue detection is marked by a green triangle. The contours were obtained from the smoothed eRASS:4 0.5 - 7 keV image. _Central panel:_ DES DR2 z-band image.Circular colored markers denote counterpart candidates for 1eRASS J202039.8-621525 from the DESI Legacy Survey catalogue. They are colour-coded according to their relative probability \texttt{p\_i} of being the best counterpart. The DESI Legacy Survey source with the highest \texttt{p\_i} is spatially co-incident with the quasar coordinates. _Right panel:_  CatWISE 2020 W1 image. Among the Chandra and eROSITA X-ray detections, only the quasar is clearly detected.  (*fig:bbchandra*)

</div>
<div id="div_fig3">

<img src="tmp_2406.05118/./lxz_simple.png" alt="Fig11.1" width="50%"/><img src="tmp_2406.05118/./aoxluv_simple.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** _Left panel_: X-ray luminosity redshift distribution of X-ray luminous quasars from literature \citep[][respectively as red diamonds, blue circles and green, yellow and brown squares]{nanni17,vito19,pons20,wolf21,wolf22} and the two newly discovered quasars (displayed as pink stars). The colour gradient shows the sensitivity to a fiducial absorbed power-law with Galactic absorption of the final cumulative eRASS in the equatorial region (in fractional of sensitive area). The quasars discovered in this work lie at the luminous end of the quasar population in the early universe. We show the luminosity derived from the Chandra follow-up observation of J202040-621509. Between the eRASS1 and the Chandra observation, its luminosity has decreased by an order of magnitude. _Right panel_:  $\alpha_{OX}$-L2500 distribution of the same sample of sources. The 1$\sigma$ confidence interval of relation derived by [Nanni, et. al (2017)]() is shown by red dotted line.  The hexagonal pattern shows a lower redshift AGN sample by [ and Lusso (2016)](). The three quasars X-ray detected, newly discovered, are over-luminous in the X-ray wavebands. Following its dimming observed in the recent Chandra observation, J202040-621509 is perfectly consistent $\alpha_{OX}$-L2500 relation.  (*fig:lxaox*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.05118"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

89  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
